<a href="https://colab.research.google.com/github/ppojawa/machine-learning-bootcamp/blob/main/supervised/02_regression/04_backward_elimination_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

np.random.seed(42)
np.set_printoptions(precision=4, suppress=True)


df_raw = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ml-course/insurance.csv')
df = df_raw.copy()

df = df.drop_duplicates()

cat_cols = [col for col in df.columns if df[col].dtype == 'O']
for col in cat_cols:
    df[col] = df[col].astype('category')

df_dummies = pd.get_dummies(df, drop_first=True)

data = df_dummies.copy()
target = data.pop('charges')

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

X_train_ols = X_train.copy()
X_train_ols = X_train_ols.values
X_train_ols = sm.add_constant(X_train_ols)

max_pvalue = 1
sl = 0.05

predictors = ['const'] + list(X_train.columns)
X_selected = X_train_ols[:, [0, 1, 2, 3, 4, 5, 6, 7, 8]]


while True:
    ols = sm.OLS(endog=y_train, exog=X_selected).fit()
    print(ols.summary(xname=predictors))

    max_pvalue = max(ols.pvalues)
    if max_pvalue < sl:
        break

    for i, (pval, pred) in enumerate(zip(ols.pvalues, predictors)):
        if pval == max_pvalue:
            print(f'removing "{pred}"')
            X_selected = np.delete(X_selected, i, 1)
            predictors = predictors[:i] + predictors[i+1:]
            break



                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.730
Model:                            OLS   Adj. R-squared:                  0.728
Method:                 Least Squares   F-statistic:                     358.1
Date:                Sat, 14 Oct 2023   Prob (F-statistic):          4.94e-295
Time:                        14:27:32   Log-Likelihood:                -10831.
No. Observations:                1069   AIC:                         2.168e+04
Df Residuals:                    1060   BIC:                         2.172e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.109e+04   1114.260  